# Day12_1: 데이터 전처리 for ML (Preprocessing)

## 📚 학습 목표

**Part 1: 기초**
1. ML을 위한 데이터 준비 과정 이해하기
2. train_test_split으로 데이터 분할하기
3. 계층적 샘플링(stratify) 개념 이해하기
4. StandardScaler로 특성 표준화하기
5. MinMaxScaler로 특성 정규화하기

**Part 2: 심화**
1. 범주형 변수를 LabelEncoder/OneHotEncoder로 변환하기
2. PolynomialFeatures로 다항 특성 생성하기
3. sklearn Pipeline으로 전처리 자동화하기
4. ColumnTransformer로 복합 파이프라인 구축하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| train_test_split | 모델 성능 평가용 데이터 분리 | 80% 학습, 20% 테스트 |
| StandardScaler | 특성 스케일 통일 | 나이(0-100) vs 연봉(0-1억) |
| OneHotEncoder | 범주형 변수 처리 | 성별(남/여) -> [1,0]/[0,1] |
| Pipeline | 전처리 자동화 | 스케일링 -> 인코딩 -> 모델 학습 |

**분석가 관점**: ML 모델의 성능은 80%가 전처리에서 결정됩니다. 잘 정제된 데이터가 좋은 모델을 만듭니다!

---

# Part 1: 기초

---

## 1.1 ML 데이터 전처리 개요

### 왜 전처리가 필요한가?

머신러닝 모델은 **숫자 데이터**만 처리할 수 있습니다. 실제 데이터는:
- 결측치가 있고
- 스케일이 다르고 (나이: 0-100, 연봉: 천만~억)
- 범주형 데이터가 있습니다 (성별, 지역 등)

전처리는 이런 문제들을 해결합니다!

In [ ]:
# 필요한 라이브러리 설치 및 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# sklearn 전처리 도구
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler,
    LabelEncoder, OneHotEncoder, OrdinalEncoder,
    PolynomialFeatures
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

In [ ]:
# Titanic 데이터 로드 (seaborn 내장 또는 URL)
try:
    import seaborn as sns
    titanic = sns.load_dataset('titanic')
except:
    url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
    titanic = pd.read_csv(url)
    # 컬럼명 소문자로 통일
    titanic.columns = titanic.columns.str.lower()
    titanic = titanic.rename(columns={'sex': 'sex', 'pclass': 'pclass'})

print(f"데이터 크기: {titanic.shape}")
titanic.head()

In [ ]:
# 데이터 정보 확인
print("=" * 50)
print("데이터 타입별 컬럼 수")
print("=" * 50)
print(titanic.dtypes.value_counts())

print("\n" + "=" * 50)
print("결측치 현황")
print("=" * 50)
print(titanic.isnull().sum()[titanic.isnull().sum() > 0])

### 💡 실무 예시: ML 전처리 체크리스트

1. **결측치 처리**: 삭제 or 대치
2. **이상치 처리**: 제거 or 변환
3. **스케일링**: 수치형 특성 정규화/표준화
4. **인코딩**: 범주형 -> 숫자형
5. **특성 생성**: 파생 변수 만들기
6. **데이터 분할**: 훈련/검증/테스트

---

## 1.2 데이터 분할 (train_test_split)

### 왜 데이터를 나누나요?

- **훈련 데이터**: 모델이 학습하는 데이터
- **테스트 데이터**: 모델 성능을 평가하는 데이터

같은 데이터로 학습+평가하면 **과적합(Overfitting)** 발생!

In [ ]:
# 간단한 데이터로 분할 연습
# 특성(X)과 타겟(y) 분리
X = titanic[['age', 'fare']].dropna()  # 결측치 제거
y = titanic.loc[X.index, 'survived']

print(f"전체 데이터: {len(X)}개")
print(f"특성 shape: {X.shape}")
print(f"타겟 shape: {y.shape}")

In [ ]:
# 기본 분할: 80% 훈련, 20% 테스트
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,      # 테스트 비율
    random_state=42     # 재현성을 위한 시드
)

print(f"훈련 데이터: {len(X_train)}개 ({len(X_train)/len(X)*100:.1f}%)")
print(f"테스트 데이터: {len(X_test)}개 ({len(X_test)/len(X)*100:.1f}%)")

In [ ]:
# 타겟 분포 확인 (불균형 체크)
print("전체 데이터 생존율:")
print(y.value_counts(normalize=True))

print("\n훈련 데이터 생존율:")
print(y_train.value_counts(normalize=True))

print("\n테스트 데이터 생존율:")
print(y_test.value_counts(normalize=True))

### 계층적 샘플링 (stratify)

타겟 클래스 비율을 유지하면서 분할합니다.

In [ ]:
# stratify 옵션으로 클래스 비율 유지
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # 타겟 비율 유지
)

print("=== stratify 적용 후 ===")
print(f"전체 생존율: {y.mean():.4f}")
print(f"훈련 생존율: {y_train_s.mean():.4f}")
print(f"테스트 생존율: {y_test_s.mean():.4f}")

In [ ]:
# 시각화: stratify 전후 비교
fig = make_subplots(rows=1, cols=2, subplot_titles=['일반 분할', 'Stratify 분할'])

# 일반 분할
fig.add_trace(
    go.Bar(x=['전체', '훈련', '테스트'],
           y=[y.mean(), y_train.mean(), y_test.mean()],
           marker_color=['blue', 'green', 'orange'],
           text=[f'{v:.3f}' for v in [y.mean(), y_train.mean(), y_test.mean()]],
           textposition='outside'),
    row=1, col=1
)

# Stratify 분할
fig.add_trace(
    go.Bar(x=['전체', '훈련', '테스트'],
           y=[y.mean(), y_train_s.mean(), y_test_s.mean()],
           marker_color=['blue', 'green', 'orange'],
           text=[f'{v:.3f}' for v in [y.mean(), y_train_s.mean(), y_test_s.mean()]],
           textposition='outside'),
    row=1, col=2
)

fig.update_layout(title='데이터 분할 방식 비교: 생존율', showlegend=False, height=400)
fig.update_yaxes(range=[0, 0.5])
fig.show()

### 💡 실무 예시: 불균형 데이터 분할

- 사기 탐지 (사기 1%, 정상 99%)
- 질병 진단 (양성 5%, 음성 95%)
- 고객 이탈 (이탈 10%, 유지 90%)

이런 경우 **반드시 stratify** 사용!

---

## 1.3 교차 검증 (Cross-Validation)

한 번 분할로는 우연히 좋은/나쁜 결과가 나올 수 있습니다.
**K-Fold 교차 검증**으로 여러 번 평가합니다.

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

# 5-Fold 교차 검증 예시
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

print("5-Fold 분할 결과:")
for fold, (train_idx, val_idx) in enumerate(kfold.split(X), 1):
    print(f"Fold {fold}: 훈련 {len(train_idx)}개, 검증 {len(val_idx)}개")

In [ ]:
# Stratified K-Fold: 클래스 비율 유지
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Stratified 5-Fold 분할 - 각 Fold별 생존율:")
for fold, (train_idx, val_idx) in enumerate(skfold.split(X, y), 1):
    train_ratio = y.iloc[train_idx].mean()
    val_ratio = y.iloc[val_idx].mean()
    print(f"Fold {fold}: 훈련 생존율 {train_ratio:.3f}, 검증 생존율 {val_ratio:.3f}")

---

## 1.4 특성 스케일링 (Feature Scaling)

### 왜 스케일링이 필요한가?

| 특성 | 범위 | 문제점 |
|------|------|--------|
| 나이 | 0-100 | 작은 값 |
| 연봉 | 0-1억 | 큰 값 |
| 키 | 140-200 | 중간 값 |

스케일이 다르면 **큰 값의 특성이 모델에 과도한 영향**을 줍니다.

In [ ]:
# Titanic의 age와 fare 비교
print("원본 데이터 통계:")
print(titanic[['age', 'fare']].describe())

In [ ]:
# 스케일 차이 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['Age 분포', 'Fare 분포'])

fig.add_trace(go.Histogram(x=titanic['age'].dropna(), name='Age', nbinsx=30), row=1, col=1)
fig.add_trace(go.Histogram(x=titanic['fare'].dropna(), name='Fare', nbinsx=30), row=1, col=2)

fig.update_layout(title='특성별 스케일 차이', showlegend=False, height=350)
fig.show()

### StandardScaler (표준화)

- **공식**: z = (x - mean) / std
- **결과**: 평균 0, 표준편차 1
- **적합**: 정규분포 가정 모델 (로지스틱 회귀, SVM)

In [ ]:
# StandardScaler 적용
from sklearn.preprocessing import StandardScaler

# 샘플 데이터 준비 (결측치 제거)
sample_data = titanic[['age', 'fare']].dropna()

# 스케일러 생성 및 적용
scaler_std = StandardScaler()
scaled_std = scaler_std.fit_transform(sample_data)

# DataFrame으로 변환
scaled_std_df = pd.DataFrame(scaled_std, columns=['age_scaled', 'fare_scaled'])

print("StandardScaler 적용 후:")
print(scaled_std_df.describe().round(3))

### MinMaxScaler (정규화)

- **공식**: x_scaled = (x - min) / (max - min)
- **결과**: 0 ~ 1 사이 값
- **적합**: 신경망, 이미지 데이터

In [ ]:
# MinMaxScaler 적용
from sklearn.preprocessing import MinMaxScaler

scaler_mm = MinMaxScaler()
scaled_mm = scaler_mm.fit_transform(sample_data)

scaled_mm_df = pd.DataFrame(scaled_mm, columns=['age_scaled', 'fare_scaled'])

print("MinMaxScaler 적용 후:")
print(scaled_mm_df.describe().round(3))

### RobustScaler (이상치 강건)

- **공식**: x_scaled = (x - median) / IQR
- **결과**: 중앙값 0, IQR 1
- **적합**: 이상치가 많은 데이터

In [ ]:
# RobustScaler 적용
from sklearn.preprocessing import RobustScaler

scaler_robust = RobustScaler()
scaled_robust = scaler_robust.fit_transform(sample_data)

scaled_robust_df = pd.DataFrame(scaled_robust, columns=['age_scaled', 'fare_scaled'])

print("RobustScaler 적용 후:")
print(scaled_robust_df.describe().round(3))

In [ ]:
# 세 스케일러 비교 시각화
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=['원본', 'StandardScaler', 'MinMaxScaler', 'RobustScaler'])

# 원본
fig.add_trace(go.Scatter(x=sample_data['age'], y=sample_data['fare'],
                         mode='markers', marker=dict(size=5, opacity=0.5)), row=1, col=1)

# StandardScaler
fig.add_trace(go.Scatter(x=scaled_std_df['age_scaled'], y=scaled_std_df['fare_scaled'],
                         mode='markers', marker=dict(size=5, opacity=0.5)), row=1, col=2)

# MinMaxScaler
fig.add_trace(go.Scatter(x=scaled_mm_df['age_scaled'], y=scaled_mm_df['fare_scaled'],
                         mode='markers', marker=dict(size=5, opacity=0.5)), row=2, col=1)

# RobustScaler
fig.add_trace(go.Scatter(x=scaled_robust_df['age_scaled'], y=scaled_robust_df['fare_scaled'],
                         mode='markers', marker=dict(size=5, opacity=0.5)), row=2, col=2)

fig.update_layout(title='스케일러별 변환 결과 비교', showlegend=False, height=600)
fig.show()

### 💡 실무 예시: 스케일러 선택 기준

| 상황 | 추천 스케일러 | 이유 |
|------|-------------|------|
| 정규분포 가정 모델 | StandardScaler | 평균 0, 표준편차 1 |
| 신경망, 이미지 | MinMaxScaler | 0-1 범위 필요 |
| 이상치 많음 | RobustScaler | 중앙값/IQR 기반 |

### ⚠️ 주의: Data Leakage 방지

**절대 테스트 데이터로 fit 하면 안 됩니다!**

In [ ]:
# 올바른 방법: 훈련 데이터로만 fit
X_train_demo = sample_data.iloc[:600]
X_test_demo = sample_data.iloc[600:]

scaler = StandardScaler()

# 훈련 데이터로 fit + transform
X_train_scaled = scaler.fit_transform(X_train_demo)

# 테스트 데이터는 transform만!
X_test_scaled = scaler.transform(X_test_demo)

print("훈련 데이터 평균:", X_train_scaled.mean(axis=0).round(3))
print("테스트 데이터 평균:", X_test_scaled.mean(axis=0).round(3))  # 정확히 0이 아님

---

# Part 2: 심화

---

## 2.1 범주형 변수 인코딩

### LabelEncoder (순서형/타겟용)

문자열을 정수로 변환합니다. **타겟 변수**나 **순서가 있는** 범주에 사용.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 성별 인코딩
le = LabelEncoder()
sex_encoded = le.fit_transform(titanic['sex'])

print("클래스 목록:", le.classes_)
print("인코딩 결과 (처음 10개):", sex_encoded[:10])

# 역변환
print("역변환:", le.inverse_transform([0, 1, 0]))

### OneHotEncoder (명목형 특성용)

순서가 없는 범주형 변수는 **원-핫 인코딩**을 사용합니다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 승선 항구 원-핫 인코딩
embarked_data = titanic[['embarked']].dropna()

ohe = OneHotEncoder(sparse_output=False, drop='first')  # 다중공선성 방지
embarked_encoded = ohe.fit_transform(embarked_data)

print("카테고리:", ohe.categories_)
print("\n인코딩 결과 (처음 5개):")
print(embarked_encoded[:5])

In [ ]:
# pandas get_dummies: 더 간단한 방법
embarked_dummies = pd.get_dummies(titanic['embarked'], prefix='embarked', drop_first=True)
print(embarked_dummies.head())

### 💡 실무 예시: 인코딩 선택 기준

| 변수 유형 | 예시 | 추천 방법 |
|----------|------|----------|
| 순서형 | 등급(낮음/중간/높음) | OrdinalEncoder |
| 명목형 | 색상(빨강/파랑/녹색) | OneHotEncoder |
| 이진형 | 성별(남/여) | LabelEncoder |
| 타겟 | 생존(0/1) | LabelEncoder |

---

## 2.2 특성 엔지니어링

### PolynomialFeatures (다항 특성)

특성 간의 **상호작용**과 **다항 관계**를 포착합니다.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# 샘플 데이터
X_sample = np.array([[2, 3], [4, 5], [6, 7]])
print("원본:\n", X_sample)

# 2차 다항 특성 생성
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_sample)

print("\n특성 이름:", poly.get_feature_names_out(['x1', 'x2']))
print("\n다항 특성:\n", X_poly)

In [ ]:
# 상호작용 특성만 (degree 없이)
poly_interact = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interact = poly_interact.fit_transform(X_sample)

print("상호작용 특성 이름:", poly_interact.get_feature_names_out(['x1', 'x2']))
print("결과:\n", X_interact)

### 로그 변환

**오른쪽 꼬리가 긴 분포**를 정규분포에 가깝게 만듭니다.

In [ ]:
# Fare의 로그 변환
fare_data = titanic['fare'].dropna()
fare_log = np.log1p(fare_data)  # log(1+x)로 0 처리

# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['원본 Fare', '로그 변환 Fare'])

fig.add_trace(go.Histogram(x=fare_data, nbinsx=50), row=1, col=1)
fig.add_trace(go.Histogram(x=fare_log, nbinsx=50), row=1, col=2)

fig.update_layout(title='로그 변환 효과', showlegend=False, height=350)
fig.show()

### 💡 실무 예시: Titanic 파생 변수

- **가족 크기**: sibsp + parch + 1
- **혼자 탑승**: 가족 크기 == 1
- **호칭 추출**: 이름에서 Mr/Mrs/Miss 등

In [ ]:
# Titanic 파생 변수 생성
titanic_fe = titanic.copy()

# 가족 크기
titanic_fe['family_size'] = titanic_fe['sibsp'] + titanic_fe['parch'] + 1

# 혼자 탑승 여부
titanic_fe['is_alone'] = (titanic_fe['family_size'] == 1).astype(int)

# 요금 구간 (저렴/보통/비쌈)
titanic_fe['fare_band'] = pd.qcut(titanic_fe['fare'].fillna(titanic_fe['fare'].median()),
                                   q=4, labels=['저렴', '보통', '비쌈', '프리미엄'])

print(titanic_fe[['sibsp', 'parch', 'family_size', 'is_alone', 'fare', 'fare_band']].head(10))

---

## 2.3 sklearn Pipeline 구축

**Pipeline**으로 여러 전처리 단계를 **하나의 객체**로 묶습니다.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# 수치형 파이프라인
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # 결측치 -> 중앙값
    ('scaler', StandardScaler())                     # 표준화
])

# 적용
numeric_features = titanic[['age', 'fare']]
numeric_transformed = numeric_pipeline.fit_transform(numeric_features)

print("파이프라인 적용 결과:")
print(pd.DataFrame(numeric_transformed, columns=['age', 'fare']).describe().round(3))

In [ ]:
# 범주형 파이프라인
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # 최빈값
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# 적용
cat_features = titanic[['sex', 'embarked']]
cat_transformed = categorical_pipeline.fit_transform(cat_features)

print("범주형 파이프라인 결과 shape:", cat_transformed.shape)
print("처음 5행:\n", cat_transformed[:5])

### ColumnTransformer: 복합 파이프라인

**수치형과 범주형을 동시에 처리**합니다.

In [ ]:
from sklearn.compose import ColumnTransformer

# 수치형/범주형 컬럼 정의
numeric_cols = ['age', 'fare']
categorical_cols = ['sex', 'embarked', 'pclass']

# ColumnTransformer 생성
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_cols),
        ('cat', categorical_pipeline, categorical_cols)
    ],
    remainder='drop'  # 나머지 컬럼 제거
)

# 전체 데이터 변환
X_transformed = preprocessor.fit_transform(titanic)

print(f"변환 전 shape: {titanic.shape}")
print(f"변환 후 shape: {X_transformed.shape}")

In [ ]:
# 변환된 특성 이름 확인
feature_names = preprocessor.get_feature_names_out()
print("변환된 특성들:")
for i, name in enumerate(feature_names):
    print(f"  {i}: {name}")

### 💡 실무 예시: 완전한 ML 파이프라인

In [ ]:
from sklearn.linear_model import LogisticRegression

# 전처리 + 모델을 하나의 파이프라인으로!
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# 데이터 준비
X = titanic[numeric_cols + categorical_cols]
y = titanic['survived']

# 결측치 있는 타겟 제거
mask = y.notna()
X = X[mask]
y = y[mask]

# 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 학습 및 평가
full_pipeline.fit(X_train, y_train)
score = full_pipeline.score(X_test, y_test)

print(f"테스트 정확도: {score:.4f}")

In [ ]:
# 교차 검증으로 더 신뢰성 있는 평가
cv_scores = cross_val_score(full_pipeline, X, y, cv=5, scoring='accuracy')

print(f"5-Fold CV 결과:")
print(f"  각 Fold: {cv_scores.round(4)}")
print(f"  평균: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 데이터 분할 ⭐

**문제**: 다음 데이터를 70% 훈련, 30% 테스트로 분할하세요.

```python
X = np.arange(100).reshape(50, 2)
y = np.random.randint(0, 2, 50)
```

**요구사항**: random_state=42, stratify 적용

In [ ]:
X = np.arange(100).reshape(50, 2)
y = np.random.randint(0, 2, 50)

# 여기에 코드를 작성하세요


### Q2. StandardScaler 적용 ⭐

**문제**: 다음 데이터에 StandardScaler를 적용하고, 변환 후 평균과 표준편차를 확인하세요.

```python
data = np.array([[10, 1000], [20, 2000], [30, 3000], [40, 4000]])
```

In [ ]:
data = np.array([[10, 1000], [20, 2000], [30, 3000], [40, 4000]])

# 여기에 코드를 작성하세요


### Q3. MinMaxScaler 범위 지정 ⭐⭐

**문제**: MinMaxScaler를 사용하여 데이터를 -1 ~ 1 범위로 변환하세요.

```python
values = np.array([[0], [25], [50], [75], [100]])
```

In [ ]:
values = np.array([[0], [25], [50], [75], [100]])

# 여기에 코드를 작성하세요


### Q4. LabelEncoder 적용 ⭐⭐

**문제**: 다음 등급 데이터를 숫자로 인코딩하고, 역변환도 해보세요.

```python
grades = ['Bronze', 'Silver', 'Gold', 'Silver', 'Bronze', 'Gold']
```

In [ ]:
grades = ['Bronze', 'Silver', 'Gold', 'Silver', 'Bronze', 'Gold']

# 여기에 코드를 작성하세요


### Q5. OneHotEncoder 적용 ⭐⭐

**문제**: 다음 색상 데이터를 원-핫 인코딩하세요.

```python
colors = np.array([['Red'], ['Blue'], ['Green'], ['Blue'], ['Red']])
```

In [ ]:
colors = np.array([['Red'], ['Blue'], ['Green'], ['Blue'], ['Red']])

# 여기에 코드를 작성하세요


### Q6. PolynomialFeatures 생성 ⭐⭐⭐

**문제**: 2개의 특성에서 2차 다항 특성을 생성하고, 특성 이름을 출력하세요.

```python
X = np.array([[1, 2], [3, 4], [5, 6]])
```

**요구사항**: bias 항 제외

In [ ]:
X = np.array([[1, 2], [3, 4], [5, 6]])

# 여기에 코드를 작성하세요


### Q7. 수치형 Pipeline 구축 ⭐⭐⭐

**문제**: 결측치 대치(중앙값) -> 표준화 파이프라인을 만들고 적용하세요.

```python
data_with_nan = np.array([[1, 10], [2, np.nan], [np.nan, 30], [4, 40]])
```

In [ ]:
data_with_nan = np.array([[1, 10], [2, np.nan], [np.nan, 30], [4, 40]])

# 여기에 코드를 작성하세요


### Q8. K-Fold 교차 검증 ⭐⭐⭐⭐

**문제**: Titanic 데이터로 5-Fold 교차 검증을 수행하고, 각 Fold의 정확도를 출력하세요.

**요구사항**:
- age, fare 특성 사용 (결측치 제거)
- LogisticRegression 모델
- StratifiedKFold 사용

In [ ]:
# Titanic 데이터 준비
X_cv = titanic[['age', 'fare']].dropna()
y_cv = titanic.loc[X_cv.index, 'survived']

# 여기에 코드를 작성하세요


### Q9. ColumnTransformer 구축 ⭐⭐⭐⭐

**문제**: 다음 데이터프레임에 대해 수치형/범주형을 동시에 처리하는 ColumnTransformer를 만드세요.

```python
df = pd.DataFrame({
    'age': [25, 30, np.nan, 40],
    'salary': [50000, 60000, 70000, np.nan],
    'department': ['IT', 'HR', 'IT', 'Sales'],
    'gender': ['M', 'F', 'M', 'F']
})
```

**요구사항**:
- 수치형: 중앙값 대치 + 표준화
- 범주형: 최빈값 대치 + 원-핫 인코딩

In [ ]:
df = pd.DataFrame({
    'age': [25, 30, np.nan, 40],
    'salary': [50000, 60000, 70000, np.nan],
    'department': ['IT', 'HR', 'IT', 'Sales'],
    'gender': ['M', 'F', 'M', 'F']
})

# 여기에 코드를 작성하세요


### Q10. 종합: Titanic ML 파이프라인 ⭐⭐⭐⭐⭐

**문제**: Titanic 데이터로 완전한 ML 파이프라인을 구축하고, 테스트 정확도를 출력하세요.

**요구사항**:
1. 사용 특성: age, fare, pclass, sex, embarked
2. 수치형(age, fare): 중앙값 대치 + StandardScaler
3. 범주형(pclass, sex, embarked): 최빈값 대치 + OneHotEncoder
4. 모델: LogisticRegression
5. 데이터 분할: 80/20, stratify, random_state=42
6. 5-Fold 교차 검증 점수도 출력

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 함수/클래스 | 핵심 파라미터 | 사용 시점 |
|------|------------|--------------|----------|
| 데이터 분할 | train_test_split | test_size, stratify, random_state | 모델 평가 준비 |
| 교차 검증 | KFold, StratifiedKFold | n_splits, shuffle | 신뢰성 있는 평가 |
| 표준화 | StandardScaler | - | 평균 0, 표준편차 1 |
| 정규화 | MinMaxScaler | feature_range | 0-1 범위 변환 |
| 이상치 강건 | RobustScaler | - | 이상치 많은 데이터 |

### Part 2: 심화 핵심 요약

| 개념 | 함수/클래스 | 핵심 파라미터 | 사용 시점 |
|------|------------|--------------|----------|
| 레이블 인코딩 | LabelEncoder | - | 타겟 변수, 이진 범주 |
| 원-핫 인코딩 | OneHotEncoder | drop, handle_unknown | 명목형 특성 |
| 다항 특성 | PolynomialFeatures | degree, interaction_only | 비선형 관계 포착 |
| 파이프라인 | Pipeline | steps | 전처리 자동화 |
| 복합 변환 | ColumnTransformer | transformers | 수치형+범주형 동시 처리 |

### 💡 실무 팁

1. **Data Leakage 주의**: 테스트 데이터로 절대 fit 하지 마세요!
2. **stratify 사용**: 불균형 데이터는 반드시 계층적 샘플링
3. **Pipeline 활용**: 전처리와 모델을 묶어 실수 방지
4. **교차 검증 필수**: 한 번 분할보다 여러 번 평가가 신뢰성 있음
5. **스케일러 선택**: 데이터 분포와 모델 특성에 맞게 선택